# Face Detection

In [154]:
#import necessary modules

import numpy as np
import pandas as pd
import matplotlib as plt
import cv2
import os

## Load images from dir


In [155]:
#images path
img_dir = 'train_images'
# Path to the folder containing test images
test_img_dir = 'test_images'
# Create a mapping from folder names to integer labels
label_map = {'wmaathai': 1, 'others': 0}

## Train FaceRecognizer Model

In [156]:
# initialize empty lists for training images and image-labels
faces = []
ids = []


In [157]:
# Load Haar Cascade classifiers for frontal and side faces
front_face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
side_face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_profileface.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

In [158]:
# Set to track processed face coordinates (to avoid duplicates)
processed_faces_coordinates = set()

In [159]:
# train FaceRecognizer model
for root, dirs, files in os.walk(img_dir):
    for filename in files:
    
        # Check if the file is an image (JPG, PNG, JPEG)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(root, filename)  # Get full image path relative to root
        
            # Read the image
            img = cv2.imread(image_path)

            # Skip invalid images
            if img is None:
                print(f"Failed to load image {image_path}")
                continue

            # Convert the image to grayscale (Haar cascades work with grayscale images)
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Detect frontal faces
            front_faces = front_face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            # Detect side faces
            side_faces = side_face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            print(f"Detected {len(front_faces)} front face(s) and {len(side_faces)} side face(s) in {image_path}")

            #skip invalid images
            if img is None:
                print(f"Failed to load image {image_path}")
                continue

            # Combine both frontal and side faces into one list only if they are valid
            all_faces = []
            if len(front_faces) > 0:
                all_faces.extend(front_faces)
            if len(side_faces) > 0:
                all_faces.extend(side_faces)
            
            # Process each detected face
            for (x, y, w, h) in all_faces:
                if (x, y, w, h) in processed_faces_coordinates:
                    continue  # Skip if face already processed

                processed_faces_coordinates.add((x, y, w, h))  # Add coordinates to avoid duplicate processing
                face_region = gray_img[y:y + h, x:x + w]  # Extract face region

                # Detect eyes within the detected face region
                eyes = eye_cascade.detectMultiScale(face_region, scaleFactor=1.1, minNeighbors=3, minSize=(30, 30))

                if len(eyes) > 0:  # If eyes are detected, process this face
                    faces.append(face_region)  # Add the face to the list
                    label = os.path.basename(root)  # Folder name as the label
                    if label in label_map:
                        ids.append(label_map[label])  # Add corresponding label


Detected 1 front face(s) and 1 side face(s) in train_images\others\1.jpg
Detected 0 front face(s) and 0 side face(s) in train_images\others\11.jpg
Detected 1 front face(s) and 0 side face(s) in train_images\others\12.jpg
Detected 0 front face(s) and 0 side face(s) in train_images\others\13.jpg
Detected 0 front face(s) and 1 side face(s) in train_images\others\14.jpg
Detected 1 front face(s) and 0 side face(s) in train_images\others\15.jpg
Detected 1 front face(s) and 0 side face(s) in train_images\others\16.jpg
Detected 1 front face(s) and 1 side face(s) in train_images\others\18.jpg
Detected 0 front face(s) and 0 side face(s) in train_images\others\19.jpg
Detected 1 front face(s) and 0 side face(s) in train_images\others\2.jpg
Detected 1 front face(s) and 0 side face(s) in train_images\others\20.jpg
Detected 0 front face(s) and 0 side face(s) in train_images\others\21.jpg
Detected 0 front face(s) and 0 side face(s) in train_images\others\22.jpg
Detected 1 front face(s) and 1 side face

## Face Detection
-consider if a face is not detected

In [160]:
# Check if faces and ids are not empty before training
if len(faces) > 0 and len(ids) > 0:
    # Create and train the recognizer
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.train(faces, np.array(ids))  # Train the recognizer with the faces and corresponding ids

    # Save the trained model
    recognizer.save('./trained/trained_model.yml')
    print("Model trained and saved successfully.")
else:
    print("No faces detected, model training skipped.")

Model trained and saved successfully.


In [161]:
# Train the recognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.train(faces, np.array(ids))


In [162]:
# Save the trained model
recognizer.save('trained/trained_model.yml')
print("Model trained and saved successfully.")


Model trained and saved successfully.


# Testing

In [163]:
# Load the trained model for face recognition
recognizer.read('trained/trained_model.yml')

In [164]:
# Loop through the test images
for test_image_name in os.listdir(test_img_dir):
    test_image_path = os.path.join(test_img_dir, test_image_name)

    test_img = cv2.imread(test_image_path)
    if test_img is None:
        print(f"Failed to load image {test_image_path}")
        continue  # Skip invalid images

    gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)  # Convert test image to grayscale for face detection

    # Detect frontal and side faces
    front_faces = front_face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    side_faces = side_face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Combine both frontal and side faces into one list
    all_faces = []
    if len(front_faces) > 0:
        all_faces.extend(front_faces)
    if len(side_faces) > 0:
        all_faces.extend(side_faces)

    # Variable to track if a recognized face is found
    is_wmaathai = False
    print(f"Processing image: {test_image_name}")

    # Process each detected face
    for (x, y, w, h) in all_faces:
        face_region = gray_img[y:y + h, x:x + w]  # Extract the face region

        # Detect eyes within the face region
        eyes = eye_cascade.detectMultiScale(face_region, scaleFactor=1.1, minNeighbors=3, minSize=(30, 30))

        if len(eyes) > 0:  # If eyes are detected, proceed with face recognition
            # Recognize the face using the trained recognizer
            label, confidence = recognizer.predict(face_region)

            # Log the label and confidence for debugging
            print(f"  Detected face with label: {label}, confidence: {confidence}")

            # Check if the confidence value is good (low confidence means a good match)
            if confidence < 100:  # Low confidence means a good match
                if label == 1:
                    is_wmaathai = True
                    text = f"{test_image_name}: wmaathai"
                    color = (0, 255, 0)  # Green text for 'wmaathai'
                else:
                    text = f"{test_image_name}: not wmaathai"
                    color = (0, 0, 255)  # Red text for 'not wmaathai'
            else:
                # If the confidence is too high, mark the face as unknown
                text = f"{test_image_name}: unknown"
                color = (0, 0, 255)  # Red text for 'unknown'

            # Draw rectangle around the face
            cv2.rectangle(test_img, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # Put text on the image
            cv2.putText(test_img, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # If no faces were detected or recognized, output unknown
    if len(all_faces) == 0:
        print(f"No faces detected in {test_image_name}")
        text = f"{test_image_name}: unknown"
        color = (0, 0, 255)
        cv2.putText(test_img, text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Convert the image to RGB (from BGR, OpenCV uses BGR format)
    test_img_rgb = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)

    # Display the image using matplotlib
    plt.imshow(test_img_rgb)
    plt.axis('off')  # Hide axes
    plt.title(f"Test Image: {test_image_name}")  # Title with the image name
    plt.show()


Processing image: 1.jpg


AttributeError: module 'matplotlib' has no attribute 'imshow'

In [153]:
# Load the trained model for face recognition
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trained/trained_model.yml')

# Initialize Haar Cascade classifiers for frontal and side faces
front_face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
side_face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_profileface.xml')

# Set to track processed face coordinates (to avoid duplicates)
processed_faces_coordinates = set()

# Loop through the test images in the test_img_dir
for test_image_name in os.listdir(test_img_dir):
    test_image_path = os.path.join(test_img_dir, test_image_name)

    # Read the test image
    test_img = cv2.imread(test_image_path)
    if test_img is None:
        print(f"Failed to load image {test_image_path}")
        continue  # Skip invalid images

    # Convert the test image to grayscale for face detection
    gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

    # Detect faces using both frontal and side face classifiers
    front_faces = front_face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    side_faces = side_face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Variable to track if a recognized face is found
    is_wmaathai = False
    print(f"Processing image: {test_image_name}")

    # Process each detected front face
    for (x, y, w, h) in front_faces:
        # Check if this face has already been processed (duplicate detection)
        if (x, y, w, h) in processed_faces_coordinates:
            continue  # Skip if face already processed

        # Add the face coordinates to the set to avoid duplicate processing
        processed_faces_coordinates.add((x, y, w, h))

        # Extract face region from grayscale image
        face_region = gray_img[y:y + h, x:x + w]

        # Recognize the face using the trained recognizer
        label, confidence = recognizer.predict(face_region)

        # Log the label and confidence for debugging
        print(f"  Detected front face with label: {label}, confidence: {confidence}")

        # Adjusted confidence check for wmaathai or others
        if confidence < 100:  # If confidence is below 100, consider it as a recognized face
            if label == 1:  # Assuming 1 corresponds to "wmaathai"
                is_wmaathai = True
                message = "is wmaathai (front face)"
            else:
                message = "is not wmaathai (front face)"
        else:
            # If confidence is 100 or higher, mark the face as unknown
            message = "front face is unknown (confidence too high)"

        # Display the result on the image
        cv2.putText(test_img, message, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        cv2.rectangle(test_img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Process each detected side face
    for (x, y, w, h) in side_faces:
        # Check if this face has already been processed (duplicate detection)
        if (x, y, w, h) in processed_faces_coordinates:
            continue  # Skip if face already processed

        # Add the face coordinates to the set to avoid duplicate processing
        processed_faces_coordinates.add((x, y, w, h))

        # Extract face region from grayscale image
        face_region = gray_img[y:y + h, x:x + w]

        # Recognize the face using the trained recognizer
        label, confidence = recognizer.predict(face_region)

        # Log the label and confidence for debugging
        print(f"  Detected side face with label: {label}, confidence: {confidence}")

        # Adjusted confidence check for wmaathai or others
        if confidence < 100:  # If confidence is below 100, consider it as a recognized face
            if label == 1:  # Assuming 1 corresponds to "wmaathai"
                is_wmaathai = True
                message = "is wmaathai (side face)"
            else:
                message = "is not wmaathai (side face)"
        else:
            # If confidence is 100 or higher, mark the face as unknown
            message = "side face is unknown (confidence too high)"

        # Display the result on the image
        cv2.putText(test_img, message, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        cv2.rectangle(test_img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # If no faces were detected, output unknown
    if len(front_faces) == 0 and len(side_faces) == 0:
        print(f"No faces detected in {test_image_name}")

    # Display the image using matplotlib
    plt.imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for correct display
    plt.title(f"Result: {message}")
    plt.axis('off')  # Hide the axis
    plt.show()


Processing image: 1.jpg
  Detected front face with label: 0, confidence: 98.40679446849418


AttributeError: module 'matplotlib' has no attribute 'imshow'

In [ ]:
# Loop through the test images in the test_img_dir
for test_image_name in os.listdir(test_img_dir):
    test_image_path = os.path.join(test_img_dir, test_image_name)

    # Read the test image
    test_img = cv2.imread(test_image_path)
    if test_img is None:
        print(f"Failed to load image {test_image_path}")
        continue  # Skip invalid images

    # Convert the test image to grayscale for face detection
    gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

    # Detect faces using both frontal and side face classifiers
    front_faces = front_face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    side_faces = side_face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Variable to track if a recognized face is found
    is_wmaathai = False
    print(f"Processing image: {test_image_name}")

    # Process each detected front face
    for (x, y, w, h) in front_faces:
        # Check if this face has already been processed (duplicate detection)
        if (x, y, w, h) in processed_faces_coordinates:
            continue  # Skip if face already processed

        # Add the face coordinates to the set to avoid duplicate processing
        processed_faces_coordinates.add((x, y, w, h))

        # Extract face region from grayscale image
        face_region = gray_img[y:y + h, x:x + w]

        # Recognize the face using the trained recognizer
        label, confidence = recognizer.predict(face_region)

        # Log the label and confidence for debugging
        print(f"  Detected front face with label: {label}, confidence: {confidence}")

        # Adjusted confidence check for wmaathai or others
        if confidence < 100:  # If confidence is below 100, consider it as a recognized face
            if label == 1:  # Assuming 1 corresponds to "wmaathai"
                is_wmaathai = True
                print(f"{test_image_name} is wmaathai (front face)")  # Correctly identified as wmaathai
            else:
                print(f"{test_image_name} is not wmaathai (front face)")  # Incorrectly identified as not wmaathai
        else:
            # If confidence is 100 or higher, mark the face as unknown
            print(f"{test_image_name} front face is unknown (confidence too high)")  # Too high confidence, unknown

    # Process each detected side face
    for (x, y, w, h) in side_faces:
        # Check if this face has already been processed (duplicate detection)
        if (x, y, w, h) in processed_faces_coordinates:
            continue  # Skip if face already processed

        # Add the face coordinates to the set to avoid duplicate processing
        processed_faces_coordinates.add((x, y, w, h))

        # Extract face region from grayscale image
        face_region = gray_img[y:y + h, x:x + w]

        # Recognize the face using the trained recognizer
        label, confidence = recognizer.predict(face_region)

        # Log the label and confidence for debugging
        print(f"  Detected side face with label: {label}, confidence: {confidence}")

        # Adjusted confidence check for wmaathai or others
        if confidence < 100:  # If confidence is below 100, consider it as a recognized face
            if label == 1:  # Assuming 1 corresponds to "wmaathai"
                is_wmaathai = True
                print(f"{test_image_name} is wmaathai (side face)")  # Correctly identified as wmaathai
            else:
                print(f"{test_image_name} is not wmaathai (side face)")  # Incorrectly identified as not wmaathai
        else:
            # If confidence is 100 or higher, mark the face as unknown
            print(f"{test_image_name} side face is unknown (confidence too high)")  # Too high confidence, unknown

    # If no faces were detected, output unknown
    if len(front_faces) == 0 and len(side_faces) == 0:
        print(f"No faces detected in {test_image_name}")

In [141]:
# Draw a rectangle around the face and display the label
        # color = (0, 255, 0)  # Green color for recognized faces    
#cv2.rectangle(test_img, (x, y), (x + w, y + h), color, 2)
            #cv2.putText(test_img, f'Person {label}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        #else:
            # If the confidence is high, mark the face as unknown
            #cv2.putText(test_img, 'Unknown', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # Show the result
    #cv2.imshow(f"Recognized Faces - {test_image_name}", test_img)
    # Save the result to a file instead of showing it
    #cv2.imwrite("recognized_faces.jpg", test_img)

    #cv2.waitKey(0)  # Wait for any key to close the window

#cv2.destroyAllWindows()
